In [2]:
import pandas as pd
import numpy as np
pd.set_option('display.float_format', lambda x: '%.3f' % x)

In [ ]:
planilha = pd.ExcelFile('Precificação WEG-OIW. Outubro V4.xlsx')
tabPrecificacao1 = pd.read_excel(planilha, 'precificação - OIW')
tabPrecificacao2 = pd.read_excel(planilha, 'precificação - WEG')

# cada aba em um dataframe
precificacaoDF1 = tabPrecificacao1[['Geração','Número de módulos','Inversor: Qtde.',' Inversor: Potência AC (kW) ','KIT OIW']]
precificacaoDF2 = tabPrecificacao2[['Geração','Número de módulos','Inversor: Qtde.',' Inversor: Potência AC (kW) ','KIT WEG (sem %)']]

In [ ]:
# verificar em cada linha a potencia do inversor - de 0 até 10 = monofasico - acima de 10 = trifasico
precificacaoDF1['Potencia Inversores'] = ''
inversorPotenciaDF = precificacaoDF1[' Inversor: Potência AC (kW) '].str.split('kw', expand=True)
inversorPotenciaDF[0] = pd.to_numeric(inversorPotenciaDF[0])
inversorPotenciaDF.loc[inversorPotenciaDF[0] <= 10, 1] = 'Monofasico'
inversorPotenciaDF.loc[inversorPotenciaDF[0] > 10, 1] = 'Trifasico'
precificacaoDF1['Potencia Inversores'] = inversorPotenciaDF[1]

# adicionando colunas e modificando colunas
precificacaoDF1['Marca Inversores'] = 'Growatt'
precificacaoDF1['potenciaModulos'] = 460
precificacaoDF1['potenciaInversores'] = inversorPotenciaDF[0]
precificacaoDF1
precificacaoDF1.rename(columns={'Geração':'kwh_mes', 'Número de módulos':'quantidadeModulos', 'Inversor: Qtde.':'quantidadeInversores',
       ' Inversor: Potência AC (kW) ':'modeloInversor1', 'KIT OIW':'preco_fibrocimento', 'Marca Inversores':'marcaInversores',
       'Potencia Inversores':'ligacaoInversores'}, inplace=True)

# reorganizando colunas
precificacaoDF1 = precificacaoDF1[['potenciaModulos','kwh_mes','quantidadeModulos','quantidadeInversores','modeloInversor1','marcaInversores','ligacaoInversores','potenciaInversores','preco_fibrocimento']]

In [ ]:
from distutils import errors


precificacaoDF1[['kwh_mes','preco_fibrocimento']] = precificacaoDF1[['kwh_mes','preco_fibrocimento']].apply(pd.to_numeric, errors='ignore')
precificacaoDF1.dtypes

In [82]:
# formatando para comando sql

listString = []
listSQLCommand = []
for row in precificacaoDF1.values:
    listString = []
    for column in row:
        # verificando se eh numero
        if isinstance(column, int) or isinstance(column, float):
            listString.append(str(column))
        else:
            listString.append('"' + str(column) + '"')
    listSQLCommand.append(','.join(listString))

listSQLCommand = ['(' + x + ')' for x in listSQLCommand]
listSQLCommand[0]    


'(460,688.3,10,1,"5kw - growatt","Growatt","Monofasico",5,13200.18)'